#### Naive Bayes (03.04 - 2.4.4)

- p(x1|y) X p(x2|y) .... p(xp|y) 구하는 부분

In [1]:
import numpy as np 
import pandas as pd

In [2]:
data = pd.read_csv('weather.nominal.csv')
data

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   outlook       14 non-null     object
 1    temperature  14 non-null     object
 2    humidity     14 non-null     object
 3    windy        14 non-null     bool  
 4    play         14 non-null     object
dtypes: bool(1), object(4)
memory usage: 590.0+ bytes


In [11]:
data.columns

Index(['outlook', ' temperature', ' humidity', ' windy', ' play'], dtype='object')

In [10]:
data.columns.values

array(['outlook', ' temperature', ' humidity', ' windy', ' play'],
      dtype=object)

In [18]:
# 컬럼명 앞뒤 공백 제거

names = pd.Series(data.columns.values)
names = names.str.strip()

# 확인
for name in names:
    print(name)


outlook
temperature
humidity
windy
play


In [19]:
data.columns = names
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   outlook      14 non-null     object
 1   temperature  14 non-null     object
 2   humidity     14 non-null     object
 3   windy        14 non-null     bool  
 4   play         14 non-null     object
dtypes: bool(1), object(4)
memory usage: 590.0+ bytes


In [20]:
data['windy'] = data['windy'].astype('str')
data.dtypes

outlook        object
temperature    object
humidity       object
windy          object
play           object
dtype: object

In [21]:
tabs = []
for col in data.columns[:-1]:
    temp = pd.crosstab(data['play'], data[col]) # play(y)에 따른 각 빈도수 산출
    tabs.append(temp)

joined_tab = pd.concat(tabs, axis=1)    # axis=1로 결합
joined_tab

,overcast,rainy,sunny,cool,hot,mild,high,normal,False,True
play,,,,,,,,,,
no,0,2,3,1,2,2,4,1,2,3
yes,4,3,2,3,2,4,3,6,6,3


In [22]:
# no : y = 0일 때, overcast ~ True : x1, x2, ...xp = (xi교y)
# yes : y = 1일 때, overcast ~ True : x1, x2, ...xp = (xi교y)

In [23]:
freqY = data.groupby('play').agg('size')        # groupby -> 알파벳순 정렬
freqY

play
no     5
yes    9
dtype: int64

In [24]:
data.play.value_counts()

yes    9
no     5
Name: play, dtype: int64

- p(x|y)= p(x교y)/p(y)
- 전체 crosstab을 play(yes/no) 빈도수로 나누기

In [25]:
# crosstab : 2Xn - play빈도 : 2
# np brodcasting : 마지막 축(axis) 같거나, 한쪽이 1이어야 함 

# play 빈도수를 2X1 로

In [27]:
freqY.to_frame()        # pd.Series를 df로 변환 

,0
play,
no,5
yes,9


In [29]:
freqY.shape, freqY.to_frame().shape

((2,), (2, 1))

In [30]:
Xprob = joined_tab / freqY.to_frame().values
Xprob

,overcast,rainy,sunny,cool,hot,mild,high,normal,False,True
play,,,,,,,,,,
no,0.000000,0.400000,0.600000,0.200000,0.400000,0.400000,0.800000,0.200000,0.400000,0.600000
yes,0.444444,0.333333,0.222222,0.333333,0.222222,0.444444,0.333333,0.666667,0.666667,0.333333


In [32]:
# or 
joined_tab / freqY.values.reshape((-1,1))

,overcast,rainy,sunny,cool,hot,mild,high,normal,False,True
play,,,,,,,,,,
no,0.000000,0.400000,0.600000,0.200000,0.400000,0.400000,0.800000,0.200000,0.400000,0.600000
yes,0.444444,0.333333,0.222222,0.333333,0.222222,0.444444,0.333333,0.666667,0.666667,0.333333


In [ ]:
# 스무딩 파라메터(Naive 추정 튜닝방법) alpha 
# : 모든 데이터 빈도수(crosstab)에 1을 주고 시작 / 성급한 확정결론 피할 수 있음 

In [35]:
# P(y) = prior 
yprob = freqY / freqY.sum()
yprob

play
no     0.357143
yes    0.642857
dtype: float64

In [39]:
# or 
data.play.value_counts(normalize=True)[::-1]    # normalize=True : 빈도/전체 비율 한번에

no     0.357143
yes    0.642857
Name: play, dtype: float64

#### What we want to know 
- 날씨 : Sunny, Mild, High, True일 때, 골프를 칠(Yes) 말(No)

- P(Sunny, Mild, High, True|Yes)과
- P(Sunny, Mild, High, True|No) 비교,
- 큰 값 선택

- naive 베이지안 분류기는 모든 변수 사건을 독립으로 가정
- P(Sunny, Mild, High, True|Yes) = P(Sunny|Yes) × ...P(True|Yes)

In [44]:
Xprob   # ㅠP(Xi|y)

,overcast,rainy,sunny,cool,hot,mild,high,normal,False,True
play,,,,,,,,,,
no,0.000000,0.400000,0.600000,0.200000,0.400000,0.400000,0.800000,0.200000,0.400000,0.600000
yes,0.444444,0.333333,0.222222,0.333333,0.222222,0.444444,0.333333,0.666667,0.666667,0.333333


In [45]:
yprob   # P(y)

play
no     0.357143
yes    0.642857
dtype: float64

In [47]:
prob = yprob.copy()
for col in ["sunny", "mild", "high", "True"]:
    prob *= Xprob[col].values
    
prob                                        #  prob.score가 정확한 표현 

play
no     0.041143
yes    0.007055
dtype: float64

In [50]:
yprob.index[prob.argmax()]

'no'

In [52]:
# 찐 확률
prob / prob.sum()

play
no     0.85363
yes    0.14637
dtype: float64

- [Quiz] 위 Xprob과 yprob을 이용하여 logprob을 계산해보자.

- log p(y=0) ~ Sigma log P(Xi|y=0) + log P(y=0)

In [57]:
# margina.prob(분모)는 공통, 고려하지 X

In [56]:
logprob = np.log(yprob)

for col in ["sunny", "mild", "high", "True"]:
    logprob += np.log(Xprob[col].values)

print(yprob.index[logprob.argmax()])
logprob

no


play
no    -3.190705
yes   -4.954065
dtype: float64

In [58]:
# no값이 yes 값보다 크므로 no값으로 예측O

In [ ]:
# ?? 스무딩 어디서 어떻게 해야 하는거니 ? 


#### 1) 다항분포 나이브 베이즈 (2.4.4(1))

In [60]:
# multinomial Variable 
# 빈도표가 존재하는 데이터 입력

# alpha : smoothing parameter (=1)
# fit_prior : 사전확률을 학습할 것인가? (=True)
# Class_prior : 데이터 학습 않고(False) 경험적인 값을 넣을 때 (=None)

In [62]:
01.03 (오후1) : 14:16

SyntaxError: invalid syntax (<ipython-input-62-5d5ddb4fa22f>, line 1)